In [1]:
from sklearn import preprocessing
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pandas
from sklearn.base import TransformerMixin
from collections import Counter

## Scaling

<img src="scaler.png" width="500">

#### StandardScaler

<img src="standard_scaler_1.png" width="500">

<img src="standard_scaler_2.png" width="500">

<img src="standard_scaler_3.png" width="500">

In [2]:
X_train = np.array([[ 1., -1.,  2.],
[ 2.,  0.,  0.],
[ 0.,  1., -1.]])

Давайте отмасштабируем каждый признак

In [3]:
X_scaled = preprocessing.scale(X_train)

In [4]:
X_scaled 

array([[ 0.        , -1.22474487,  1.33630621],
       [ 1.22474487,  0.        , -0.26726124],
       [-1.22474487,  1.22474487, -1.06904497]])

**Что мы ожидаем от этого кода?**

In [5]:
print(X_scaled.mean(axis=0))
print(X_scaled.std(axis=0))

[0. 0. 0.]
[1. 1. 1.]


In [6]:
scaler = preprocessing.StandardScaler().fit(X_train)
scaler

StandardScaler(copy=True, with_mean=True, with_std=True)

Параметры преобразования

In [7]:
scaler.mean_

array([1.        , 0.        , 0.33333333])

In [8]:
scaler.scale_

array([0.81649658, 0.81649658, 1.24721913])

In [9]:
X_train

array([[ 1., -1.,  2.],
       [ 2.,  0.,  0.],
       [ 0.,  1., -1.]])

In [10]:
scaler.transform(X_train)

array([[ 0.        , -1.22474487,  1.33630621],
       [ 1.22474487,  0.        , -0.26726124],
       [-1.22474487,  1.22474487, -1.06904497]])

Перепроверим свойства преобразованных данных

In [11]:
print(scaler.transform(X_train).mean(axis=0))
print(scaler.transform(X_train).std(axis=0))

[0. 0. 0.]
[1. 1. 1.]


In [12]:
scaler = preprocessing.StandardScaler(with_std=False).fit(X_train)
print(scaler.transform(X_train).mean(axis=0))
print(scaler.transform(X_train).std(axis=0))

[0.00000000e+00 0.00000000e+00 7.40148683e-17]
[0.81649658 0.81649658 1.24721913]


#### MinMaxScaler

<img src="min_max_scaler.png" width="500">

Масштабировать признаки можно по-разному, в прошлом варианте мы делали значения несмещёнными и с единичной дисперсией, теперь мы будем проектировать значения на заданный отрезок

In [13]:
min_max_scaler = preprocessing.MinMaxScaler()
min_max_scaler

MinMaxScaler(copy=True, feature_range=(0, 1))

In [14]:
min_max_scaler.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

Параметры масшбирования немного другие

In [15]:
min_max_scaler.scale_

array([0.5       , 0.5       , 0.33333333])

In [16]:
min_max_scaler.min_  

array([0.        , 0.5       , 0.33333333])

In [17]:
X_scaled = min_max_scaler.fit_transform(X_train)
X_scaled

array([[0.5       , 0.        , 1.        ],
       [1.        , 0.5       , 0.33333333],
       [0.        , 1.        , 0.        ]])

Проверяем корректность преобразования

In [18]:
print(X_scaled.min(axis=0))
print(X_scaled.max(axis=0))

[0. 0. 0.]
[1. 1. 1.]


#### MaxAbsScaler

<img src="max_abs_scaler.png" width="500">

В данном случае нормируем столбцы на наибольшее значение по модулю

In [19]:
max_abs_scaler = preprocessing.MaxAbsScaler()
max_abs_scaler

MaxAbsScaler(copy=True)

In [20]:
max_abs_scaler.fit(X_train)

MaxAbsScaler(copy=True)

In [21]:
max_abs_scaler.scale_

array([2., 1., 2.])

In [22]:
X_scaled = max_abs_scaler.fit_transform(X_train)
X_scaled

array([[ 0.5, -1. ,  1. ],
       [ 1. ,  0. ,  0. ],
       [ 0. ,  1. , -0.5]])

Минимум и максимум будут из отрезка [-1, 1]

In [23]:
print(X_scaled.min(axis=0))
print(X_scaled.max(axis=0))

[ 0.  -1.  -0.5]
[1. 1. 1.]


In [24]:
print(np.abs(X_scaled).min(axis=0))
print(np.abs(X_scaled).max(axis=0))

[0. 0. 0.]
[1. 1. 1.]


## Пример использования на датасете

Возьмём стандартный набор данных

In [25]:
data = load_breast_cancer()

In [26]:
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.8, random_state=42)

Обучим  kNN

In [27]:
algo = KNeighborsClassifier().fit(X_train, y_train)
accuracy_score(algo.predict(X_test), y_test)

0.9013157894736842

По умолчанию kNN использует $l_2$ метрику, поэтому разные мастабы признаков могут негативно сказаться на качестве, давайте попробуем исправить эту проблему

In [28]:
X_train, X_test, y_train, y_test = train_test_split(preprocessing.scale(data.data), data.target, test_size=0.8, random_state=42)

In [29]:
algo = KNeighborsClassifier().fit(X_train, y_train)
accuracy_score(algo.predict(X_test), y_test)

0.9605263157894737

**Почему так не корректно проверять?**

In [30]:
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.8, random_state=42)

In [31]:
scaler = preprocessing.StandardScaler().fit(X_train)
algo = KNeighborsClassifier().fit(scaler.transform(X_train), y_train)
accuracy_score(algo.predict(scaler.transform(X_test)), y_test)

0.956140350877193

Эта оценка более корректна, но лучше использовать kFold кроссвалидацию

Как корректно запустить с cross_val_score?

In [32]:
cross_val_score(KNeighborsClassifier(), preprocessing.scale(data.data), data.target, cv=3).mean()

0.9613292490485472

Такой вариант некорректен

Для комбинации нескольких шагов обработки данных удобно пользоваться Pipeline

In [33]:
algo = make_pipeline(preprocessing.StandardScaler(), KNeighborsClassifier())
algo

Pipeline(memory=None,
     steps=[('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('kneighborsclassifier', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'))])

In [34]:
cross_val_score(algo, data.data, data.target, cv=3).mean()

0.9630929174788824

# Normalization

Делать масштабирование можно не только в рамках одного признака, но и в рамках одного объекта

Normalizer приводит признаки каждого объекта к единичной норме по какой-то метрике, т.е. нормирует по строкам

In [35]:
X_train = np.array([[ 1., -1.,  2.],
[ 2.,  0.,  0.],
[ 0.,  1., -1.]])

In [36]:
normalizer = preprocessing.Normalizer().fit(X_train)
normalizer

Normalizer(copy=True, norm='l2')

In [37]:
X_scaled = normalizer.transform(X_train)

In [38]:
X_scaled 

array([[ 0.40824829, -0.40824829,  0.81649658],
       [ 1.        ,  0.        ,  0.        ],
       [ 0.        ,  0.70710678, -0.70710678]])

**Какие значения мы можем гарантировать, а какие нет?**

In [39]:
print(X_scaled.mean(axis=1))
print(X_scaled.std(axis=1))
print((X_scaled ** 2).sum(axis=1))

[0.27216553 0.33333333 0.        ]
[0.50917508 0.47140452 0.57735027]
[1. 1. 1.]


В pipeline можно комбинировать много шагов, необязательно два

In [40]:
cross_val_score(
    make_pipeline(preprocessing.StandardScaler(), preprocessing.Normalizer(), KNeighborsClassifier()), 
    data.data, 
    data.target, 
    cv=3
).mean()

0.9630743525480367

# Binarization

In [41]:
X_train = np.array([[ 1., -1.,  2.],
[ 2.,  0.,  0.],
[ 0.,  1., -1.]])

In [42]:
binarizer = preprocessing.Binarizer(threshold=0.).fit(X_train)
binarizer

Binarizer(copy=True, threshold=0.0)

In [43]:
binarizer.transform(X_train)

array([[1., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.]])

## Imputation of missing values

Иногда в данных присутсвуют пропуски, а для корректной работы методов их быть не должно. Sklearn предоставляет стандартные методы обработки пропусков.

In [44]:
X_train = [
    [1, 2], 
    [np.nan, 3], 
    [7, 6]
]

In [45]:
imputer = preprocessing.Imputer(missing_values='NaN', strategy='mean', axis=0).fit(X_train)
imputer

Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbose=0)

In [46]:
print(imputer.transform(X_train)) 

[[1. 2.]
 [4. 3.]
 [7. 6.]]


In [47]:
X_test = [
    [np.nan, 2], 
    [6, np.nan], 
    [7, 6]
]

In [48]:
print(imputer.transform(X_test)) 

[[4.         2.        ]
 [6.         3.66666667]
 [7.         6.        ]]


## Encoding categorical features

## One hot encoding

![](one_hot_encoding.png)

In [49]:
X_train = np.array([
    [0, 0, 3], 
    [1, 1, 0], 
    [0, 2, 1], 
    [1, 0, 2]
])

In [50]:
encoder = preprocessing.OneHotEncoder().fit(X_train)
encoder

OneHotEncoder(categorical_features='all', dtype=<class 'numpy.float64'>,
       handle_unknown='error', n_values='auto', sparse=True)

In [51]:
encoder.transform([[0, 1, 3]])

<1x9 sparse matrix of type '<class 'numpy.float64'>'
	with 3 stored elements in Compressed Sparse Row format>

In [52]:
encoder.transform([[0, 1, 3]]).todense()

matrix([[1., 0., 0., 1., 0., 0., 0., 0., 1.]])

Можем явно увеличивать количество классов

In [53]:
encoder = preprocessing.OneHotEncoder(n_values=[3, 2, 4]).fit(X_train)
encoder

OneHotEncoder(categorical_features='all', dtype=<class 'numpy.float64'>,
       handle_unknown='error', n_values=[3, 2, 4], sparse=True)

In [54]:
encoder.transform([[0, 1, 3]]).todense()

matrix([[1., 0., 0., 0., 1., 0., 0., 0., 1.]])

Но уменьшать не можем

In [55]:
encoder = preprocessing.OneHotEncoder(n_values=[2, 2, 3]).fit(X_train)
encoder

ValueError: column index exceeds matrix dimensions

Пример использования

In [56]:
df = pandas.read_csv('mushrooms.csv', header=None)

In [57]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [58]:
X, y = np.array(df.loc[:, 1:]), np.array(df.loc[:, 0])

In [59]:
X

array([['x', 's', 'n', ..., 'k', 's', 'u'],
       ['x', 's', 'y', ..., 'n', 'n', 'g'],
       ['b', 's', 'w', ..., 'n', 'n', 'm'],
       ...,
       ['f', 's', 'n', ..., 'b', 'c', 'l'],
       ['k', 'y', 'n', ..., 'w', 'v', 'l'],
       ['x', 's', 'n', ..., 'o', 'c', 'l']], dtype=object)

OneHotEncoder требует, чтобы метки были числами

In [60]:
encoder = preprocessing.OneHotEncoder().fit(X)
encoder

ValueError: could not convert string to float: 'l'

Поэтому сначала нужно преобразовать данные

In [61]:
label_encoder = preprocessing.LabelEncoder()
label_encoder

LabelEncoder()

In [62]:
for i in range(X.shape[1]):
    X[:, i] = label_encoder.fit_transform(X[:, i])
X

array([[5, 2, 4, ..., 2, 3, 5],
       [5, 2, 9, ..., 3, 2, 1],
       [0, 2, 8, ..., 3, 2, 3],
       ...,
       [2, 2, 4, ..., 0, 1, 2],
       [3, 3, 4, ..., 7, 4, 2],
       [5, 2, 4, ..., 4, 1, 2]], dtype=object)

In [63]:
encoder = preprocessing.OneHotEncoder().fit(X)
encoder

OneHotEncoder(categorical_features='all', dtype=<class 'numpy.float64'>,
       handle_unknown='error', n_values='auto', sparse=True)

In [64]:
X = encoder.fit_transform(X).todense()
X

matrix([[0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [65]:
y = np.equal(y, 'p').astype(int)
y

array([1, 0, 0, ..., 0, 1, 0])

In [66]:
cross_val_score(LogisticRegression(), X, y, cv=3).mean()

0.9054427309169989

## Mean encoding

![](mean_encoding.png)

In [67]:
X, y = np.array(df.loc[:, 1:]), np.array(df.loc[:, 0])
y = np.equal(y, 'p').astype(int)

for i in range(X.shape[1]):
    le = label_encoder.fit(X[:, i])
    X[:, i] = le.transform(X[:, i])
    for j in range(len(le.classes_)):
        indices = X[:, i] == j
        X[indices, i] = y[indices].mean()

In [68]:
cross_val_score(LogisticRegression(), X, y, cv=3).mean()

0.9087601721608912

Но, как мы помним, это некорректная оценка

Давайте честно оценим качество

In [69]:
X, y = np.array(df.loc[:, 1:]), np.array(df.loc[:, 0])
y = np.equal(y, 'p').astype(int)
for i in range(X.shape[1]):
    X[:, i] = label_encoder.fit_transform(X[:, i])

In [70]:
cross_val_score(
    make_pipeline(
        preprocessing.OneHotEncoder(),
        LogisticRegression()
    ),
    X, 
    y,
    cv=10,
).mean()

0.970210042421016

In [71]:
cross_val_score(
    make_pipeline(
        preprocessing.OneHotEncoder(),
        LogisticRegression()
    ),
    X, 
    y,
    cv=3
).mean()

ValueError: unknown categorical feature present [ 6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6
  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6
  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6
  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6
  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6
  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6
  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6
  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6
 11  8  8 11  8  8 11  3  8  3  3  3  3 11 11 11 11 11 11  8 11 11  8 11
 11  8  8  3 11  8 11  8  8 11  8 11  8 11  3 11  3  8  3 11  8 11  8 11
  8  3  8  3 11  8 11  3  8  3  8 11 11  3 11  8 11 11  3  8  3 11  8 11
  3 11 11  8 11 11  8  3  3  8  3 11 11 11  8 11  8 11 11 11  3  3  8  3
 11 11  3  8 11  8  3  8  3  8 11  8 11  3 11  8 11 11  8 11 11 11  8 11
  8

Мы не указали поведение encoder-a на ранее не наблюдаемых значениях признака, поэтому получаем ошибку, если указать поведение явно, то ошибки не будет

**Почему при cv=10 ошибки не было?**

In [72]:
cross_val_score(
    make_pipeline(
        preprocessing.OneHotEncoder(handle_unknown='ignore'),
        LogisticRegression()
    ),
    X, 
    y,
    cv=3
).mean()

0.9054427309169989

Чтобы сделать pipeline напишем собственный трансформер

Чтобы не писать лишних методов наседуемся от базового класса TransformerMixin - теперь не нужно реализовывать fit_transform

In [73]:
class MeanTransformer(TransformerMixin):
    
    def fit(self, X, y):
        self.cnt = Counter()
        for i in range(X.shape[1]):
            for j in range(np.max(X[:, i])):
                indices = X[:, i] == j
                if np.sum(indices) > 0:
                    val = y[indices].mean()
                else:
                    val = y.mean()
                self.cnt[(i, j)] = val
                
        return self
    
    def transform(self, X):
        X_new = np.copy(X)
        for i in range(X.shape[1]): 
            for j in range(np.max(X[:, i])):
                indices = X[:, i] == j
                if np.sum(indices) > 0:
                    X_new[indices, i] = self.cnt[(i, j)]
        return X_new


In [74]:
X, y = np.array(df.loc[:, 1:]), np.array(df.loc[:, 0])
y = np.equal(y, 'p').astype(int)
for i in range(X.shape[1]):
    X[:, i] = label_encoder.fit_transform(X[:, i])

In [75]:
cross_val_score(
    make_pipeline(
        MeanTransformer(),
        LogisticRegression()
    ),
    X, 
    y,
    cv=3
).mean()

0.7347445102087451

In [76]:
cross_val_score(
    make_pipeline(
        MeanTransformer(),
        LogisticRegression()
    ),
    X, 
    y,
    cv=10
).mean()

0.808425714442162

Как видите, качество заметно ниже



### Напишите MeanTransformer, который бы при обучении считал счётчики не по всей обучающей выборке, а только по предыдущим объектам - придётся написать свой метод fit_transform